In [42]:
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import folium
from geojson import Feature, FeatureCollection, Point
import json
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from folium import IFrame
from pyproj import Proj
from shapely.geometry import Polygon
pd.set_option('display.max_columns', None)

In [3]:
# https://alysivji.github.io/getting-started-with-folium.html
# https://opendata.dc.gov/datasets/294e062cdf2c48d5b9cbc374d9709bc0_2/data

In [4]:
streets = pd.read_csv("data/source data/streets.csv")
streetSegs = pd.read_csv("data/source data/streetSegs.csv")
bikeData = pd.read_csv("data/modified data/bikes.csv")

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19,20,21,22,23,24,25,26,33,37,40,41,44,52,53,55,59,61,62,72,77,78,84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [43]:
streets

,OBJECTID,ROUTEID,FROMMEASURE,TOMEASURE,ROUTENAME,ROADTYPE,SUBBLOCKKEY,TOTALTRAVELLANES,TOTALPARKINGLANES,TOTALRAISEDBUFFERS,TOTALTRAVELLANEWIDTH,TOTALCROSSSECTIONWIDTH,TOTALPARKINGLANEWIDTH,TOTALTRAVELLANESINBOUND,TOTALTRAVELLANESOUTBOUND,TOTALTRAVELLANESBIDIRECTIONAL,TOTALTRAVELLANESREVERSIBLE,SUMMARYDIRECTION,BIKELANE_PARKINGLANE_ADJACENT,BIKELANE_THROUGHLANE_ADJACENT,BIKELANE_POCKETLANE_ADJACENT,BIKELANE_CONTRAFLOW,BIKELANE_CONVENTIONAL,BIKELANE_DUAL_PROTECTED,BIKELANE_DUAL_BUFFERED,BIKELANE_PROTECTED,BIKELANE_BUFFERED,DOUBLEYELLOW_LINE,SECTIONFLAGS,LOC_ERROR,MIDMEASURE,AADT,FHWAFUNCTIONALCLASS,HPMSID,HPMSSECTIONTYPE,ID,IRI,IRI_DATE,NHSCODE,OWNERSHIP,PCI_CONDCATEGORY,PCI_LASTINSPECTED,PCI_SCORE,SIDEWALK_IB_PAVTYPE,SIDEWALK_IB_WIDTH,SIDEWALK_OB_PAVTYPE,SIDEWALK_OB_WIDTH,SPEEDLIMITS_IB,SPEEDLIMITS_IB_ALT,SPEEDLIMITS_OB,SPEEDLIMITS_OB_ALT,SUBBLOCKID,BLOCKID,BLOCKKEY,DCFUNCTIONALCLASS,NHSTYPE,QUADRANT,STREETNAME,STREETTYPE,SNOWROUTE_DPW,SNOWZONE_DPW,SNOWROUTE,SNOWSECTION,SNOWZONE,LEFTTURN_CURBLANE_EXCL,LEFTTURN_CURBLANE_EXCL_LEN,RIGHTTURN_CURBLANE_EXCL,RIGHTTURN_CURBLANE_EXCL_LEN,TOTALBIKELANES,TOTALBIKELANEWIDTH,RPPDIRECTION,RPPSIDE,SLOWSTREETINFO,TOTALRAISEDBUFFERWIDTH,AADT_YEAR,RIGHTTURN_EXCLUSIVE,LEFTTURN_EXCLUSIVE,BUSLANE_INBOUND,BUSLANE_OUTBOUND,FROMSTREET,TOSTREET,WARD_ID,SMD_ID,ANC_ID,NETWORK_LOGICAL_END,AADT_COMBINATION,AADT_COMBINATION_YEAR,AADT_SINGLE_UNIT,AADT_SINGLE_UNIT_YEAR,SHAPELEN,SHAPE
0,1693611,58013452,0.000,102.534599,Driveway-58013452,5,0563dc93f6ad6b0dda865170cfb8b7e3,0,0,0,0,0,0,0,0,0,0,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SectionType_19 Not Defined,",NO ERROR,NaN,NaN,NaN,NaN,NaN,31fa11f4-29fc-4213-8792-5b20b00dffaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58013452-14020982_58013452-14018022_58013452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Block Only,Block Only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,1693612,58012282,0.000,50.398998,Driveway-58012282,5,db06688372f061817971ad1d86a6c12a,0,0,0,0,0,0,0,0,0,0,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SectionType_19 Not Defined,",NO ERROR,NaN,NaN,NaN,NaN,NaN,ffe6ad94-cc4c-4a1a-ac95-f6edfed06a12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58012282-58001712_58002532_58012282-13046732_3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Block Only,Block Only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,1693613,58009982,0.000,50.601601,Driveway-58009982,5,a017ec180fe259a69ac8a9a4919460bb,0,0,0,0,0,0,0,0,0,0,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SectionType_19 Not Defined,",NO ERROR,NaN,NaN,NaN,NaN,NaN,84dbbbab-693c-42ed-bc56-065319031db8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58009982-11063762_58009982_58013282-58002092_5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Block Only,Block Only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,1693614,58007512,0.000,285.839691,Driveway-58007512,5,102407401665da1f7ac8623dc90d1a31,0,0,0,0,0,0,0,0,0,0,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SectionType_19 Not Defined,",NO ERROR,NaN,NaN,NaN,NaN,NaN,db39cdb2-4666-4dd4-8212-ef798c3f7429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58007512-58005692_58007512_58022852-58005692_5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Block Only,Block Only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,1693711,58005212,0.000,15.477400,Driveway-58005212,5,cdaedff3332171ceddb7f839e8dab1dc,0,0,0,0,0,0,0,0,0,0,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"SectionType_19 Not Defined,",NO ERROR,NaN,NaN,NaN,NaN,NaN,60ca9976-46e5-40d5-a77a-b2f4277de45f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58005212-58001192_58002682_58005212-58002562_5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Block Only,Block Only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
.

In [44]:
streetSegs

,STREETSEGID,FACILITYID,SOURCEID,STREETID,REGISTEREDNAME,STREETTYPE,QUADRANT,DIRECTIONALITY,SEGMENTTYPE,FROMNODEID,TONODEID,FROMADDRESSLEFTTHEO,TOADDRESSLEFTTHEO,FROMADDRESSRIGHTTHEO,TOADDRESSRIGHTTHEO,BEGINMEASURE,ENDMEASURE,UPDATETIMESTAMP,OBJECTID_1,OBJECTID,SHAPELEN
0,7581,SEGID-7581,10180320,100180,18TH,ST,NW,2,2,21821,21704,3100.0,3154.0,3101.0,3155.0,0,97.487740,2005-11-01T16:55:47.000Z,2546788,2684322,97.487740
1,5497,SEGID-5497,10180330,100180,18TH,ST,NW,2,2,21704,21810,3156.0,3198.0,3157.0,3199.0,0,97.562354,2005-11-01T16:55:47.000Z,2546789,2684323,97.562354
2,8038,SEGID-8038,10180340,100180,18TH,ST,NW,2,2,21810,18209,3200.0,3298.0,3201.0,3299.0,0,101.945998,2005-11-01T16:55:47.000Z,2546790,2684324,101.946006
3,588,SEGID-588,10180350,100180,18TH,ST,NW,2,2,18209,21785,3300.0,3334.0,3301.0,3335.0,0,101.815721,2005-11-01T16:55:47.000Z,2546791,2684325,101.815713
4,9760,SEGID-9760,10180360,100180,18TH,ST,NW,2,2,21785,15677,3336.0,3398.0,3337.0,3399.0,0,97.960598,2005-11-01T16:55:47.000Z,2546792,2684326,97.960628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13671,15525,SEGID-15429,14032251,104032,GILLIS,AVE,NW,2,2,24929,11532,0.0,0.0,0.0,0.0,0,116.428607,2015-10-06T15:03:49.000Z,2559514,2696629,116.428566
13672,15526,SEGID-15526,15970247,105970,MAURY,PL,NW,2,2,11663,24929,0.0,0.0,0.0,0.0,0,100.124889,2015-10-06T15:03:50.000Z,2559515,2696630,100.124902
13673,15509,SEGID-15509,19338010,109338,WILKES,AVE,NW,2,2,12059,11532,0.0,0.0,0.0,0.0,0,99.395468,2015-09-28T16:05:56.000Z,2559516,2696631,99.395459
13674,15529,SEGID-4075,16643000,106643,OBSERVATORY,CIR,NW,2,2,22182,24931,2.0,36.0,1.0,37.0,0,24.741446,2015-10-06T19:42:43.000Z,2559517,2696632,24.741457


In [73]:
def makeCluster(dataset, eps, minSamples): 
    accPoints = np.empty((0, 2))   

    for lat,long in zip(dataset['LATITUDE'], dataset['LONGITUDE']):
        accPoints = np.append(accPoints, np.array([[lat,long]]), axis=0)

    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=minSamples).fit(accPoints)

    clusters = db.fit_predict(accPoints)
    dataset['CLUSTER'] = clusters


    clusters = dataset['CLUSTER'].unique()   
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
    primaryFacilityTypeDict = {}
    secondaryFacilityTypeDict = {}
    primaryStreetSegIdDict = {}
    secondaryStreetSegIdDict = {}
    
    for i in clusters:
        clusterSet = dataset[(dataset["CLUSTER"] == i) & (dataset["STREETSEGID"] > 0)]
        valueCounts = clusterSet['STREETSEGID'].value_counts()
        
        if (len(valueCounts.index) > 1):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["STREETSEGID"] == int(primaryValueId)].reset_index()["REGISTEREDNAME"][0]
            
            secondaryValueId = valueCounts.index[1]
            secondaryValue = streetSegs[streetSegs["STREETSEGID"] == int(secondaryValueId)].reset_index()["REGISTEREDNAME"][0]
           
            primaryStreetDict[i] = primaryValue
            secondaryStreetDict[i] = secondaryValue
            
#             facilityResp = streets[streets["STREETSEGID"] == int(primaryValueId)]
#             if (len(facilityResp) > 0):    
#                 primaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
#                 primaryStreetSegIdDict[i] = primaryValueId
#             else:
#                 primaryFacilityTypeDict[i] = "None"
#                 primaryStreetSegIdDict[i] = "None"
#             facilityResp = streets[streets["STREETSEGID"] == int(secondaryValueId)]
#             if (len(facilityResp) > 0):  
#                 secondaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
#                 secondaryStreetSegIdDict[i] = secondaryValueId
#             else:
#                 secondaryFacilityTypeDict[i] = "None"
#                 secondaryStreetSegIdDict[i] = "None"

        elif (len(valueCounts.index) > 0):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["STREETSEGID"] == int(primaryValueId)].reset_index()["REGISTEREDNAME"][0]
            primaryStreetDict[i] = primaryValue
#             facilityResp = streets[streets["STREETSEGID"] == int(primaryValueId)]
#             if (len(facilityResp) > 0):    
#                 primaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
#                 primaryStreetSegIdDict[i] = primaryValueId
#             else:
#                 primaryFacilityTypeDict[i] = "None"
#                 primaryStreetSegIdDict[i] = "None"
                
            secondaryStreetDict[i] = "None"
#             secondaryFacilityTypeDict[i] = "None"
#             secondaryStreetSegIdDict[i] = "None"
        else:
            primaryStreetDict[i] = "None"
            secondaryStreetDict[i] = "None"
#             primaryFacilityTypeDict[i] = "None"
#             secondaryFacilityTypeDict[i] = "None"
#             primaryStreetSegIdDict[i] = "None"
#             secondaryStreetSegIdDict[i] = "None"

    primaryStreets = []
    secondaryStreets = []
#     primaryFacilityTypes = []
#     secondaryFacilityTypes = []
#     primaryStreetSegIds = []
#     secondaryStreetSegIds = []

    for i in dataset["CLUSTER"]:
        if (primaryStreetDict[i] == "None"):
            primaryStreets.append("None")
#             primaryFacilityTypes.append("None")
#             primaryStreetSegIds.append("None")
        else:
            primaryStreets.append(primaryStreetDict[i])
#             primaryFacilityTypes.append(primaryFacilityTypeDict[i])
#             primaryStreetSegIds.append(primaryStreetSegIdDict[i])
        if (secondaryStreetDict[i] == "None"):
            secondaryStreets.append("None")
#             secondaryFacilityTypes.append("None")
#             secondaryStreetSegIds.append("None")
        else:
            secondaryStreets.append(secondaryStreetDict[i])
#             secondaryFacilityTypes.append(secondaryFacilityTypeDict[i])
#             secondaryStreetSegIds.append(secondaryStreetSegIdDict[i])

    dataset['CLUSTERPRIMARYSTREET'] = primaryStreets
    dataset['CLUSTERSECONDARYSTREET'] = secondaryStreets
#     dataset['CLUSTERPRIMARYFACILITYTYPE'] = primaryFacilityTypes
#     dataset['CLUSTERSECONDARYFACILITYTYPE'] = secondaryFacilityTypes
#     dataset['CLUSTERPRIMARYSTREETSEGID'] = primaryStreetSegIds
#     dataset['CLUSTERSECONDARYSTREETSEGID'] = secondaryStreetSegIds
    
    datasetSmall = dataset[['LATITUDE','LONGITUDE','CLUSTER']]
    datasetSmall = datasetSmall[datasetSmall['CLUSTER'] != -1]
    datasetSmall.to_csv("data/cluster data/datasets/" + str(len(clusters) - 1) + "clusteredAccidentLevelData.csv")

    return dataset
    
def makeMap(dataset):
    # plotting all bike accidents
    dcMap = folium.Map(location=[38.9072, -77.0369], zoom_start=13)

    featureGroupDict = {}
        
    counter = 0
#     for lat, long, cluster, clusterprimaryStreet, clusterSecondaryStreet, fb in zip(dataset['LATITUDE'], dataset['LONGITUDE'], dataset["CLUSTER"], dataset["CLUSTERPRIMARYSTREET"], dataset["CLUSTERSECONDARYSTREET"], dataset['FATAL_BICYCLIST']):
    for lat, long, cluster, clusterprimaryStreet, clusterSecondaryStreet, fb in zip(dataset['LATITUDE'], dataset['LONGITUDE'], dataset["CLUSTER"], dataset["CLUSTERPRIMARYSTREET"], dataset["CLUSTERSECONDARYSTREET"], dataset['FATAL_BICYCLIST']):
        
        if (cluster != -1):
            if (cluster not in featureGroupDict.keys()):
                featureGroupDict[cluster] = folium.FeatureGroup(name=(str(clusterprimaryStreet) + " and " + str(clusterSecondaryStreet)))
            folium.CircleMarker(location=[lat, long], radius = 2, color = 'red').add_to(featureGroupDict[cluster])

#             if (fb > 0):
#                 print(featureGroupDict);
#                 folium.CircleMarker(location=[lat, long], radius = 2, color = 'red').add_to(featureGroupDict[cluster])
#             else:
#                 folium.CircleMarker(location=[lat, long], radius = .5, color = 'black').add_to(featureGroupDict[cluster])

        counter = counter + 1 
        
    clusters = makeHulls(dataset)
    geoString, clusters = makeGeoJson(clusters)
    
    text_file = open("data/cluster data/geoJSONs/" + str(len(clusters)) + "geoJSON.geojson", "w")
    text_file.write(geoString)
    text_file.close()
                     
    featureGroupDict["Layers"] = folium.FeatureGroup(name="Layers")
    ourLayers = folium.GeoJson(json.loads(geoString), style_function=style_function,
                               tooltip=folium.features.GeoJsonTooltip(
                                fields=['primaryStreet','secondaryStreet']
                              ))
#     ourLayers = folium.GeoJson(json.loads(geoString), style_function=style_function,
#                                tooltip=folium.features.GeoJsonTooltip(
#                                 fields=['primaryStreet','secondaryStreet','primaryFacilityType','secondaryFaciltyType','primaryStreetSegId','secondaryStreetSegId']
#                               ))
    ourLayers.add_to(featureGroupDict["Layers"]);
    
    for key in featureGroupDict.keys():
        dcMap.add_child(featureGroupDict[key])
        
    folium.map.LayerControl('topright', collapsed=False).add_to(dcMap)

    item_txt = """<br> &nbsp; {item} &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{col}"></i>"""
    html_itms = item_txt.format(item= "None" , col= "red")

    legend_html = """
     <div style="
     position: fixed; 
     bottom: 5px; left: 5px; width: 220px; height: 160px; 
     border:2px solid grey; z-index:9999; 
     background-color:white;
     opacity: .85;
     font-size:14px;
     font-weight: bold;
     ">
     <div style= "text-align:center">Street Bike Facilities</div>
     <br>
     <div><div style="margin-left:20px;">None </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:red;">  </div></div>
     <div><div style="margin-left:20px;">Shared Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:orange;">  </div></div>
     <div><div style="margin-left:20px;">Contraflow Bike Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:purple;">  </div></div>
     <div><div style="margin-left:20px;">Existing Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:blue;">  </div></div>
     <div><div style="margin-left:20px;">Climbing Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:pink;">  </div></div>
     <div><div style="margin-left:20px;">Cycle Track </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:green;">  </div></div>
      
      
      
      
      
      </div> """
    dcMap.get_root().html.add_child(folium.Element( legend_html ))
    
    framedData = pd.read_json(geoString)
#     clusters.to_csv("data/cluster data/" + str(len(framedData)) + "clustersalldata.csv")
    
    return dcMap

def makeHulls(dataset):
    features = []
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
    primaryFacilityTypeDict = {}
    secondaryFacilityTypeDict = {}
    primaryStreetSegIdDict = {}
    secondaryStreetSegIdDict = {}
    totalAccidentNumberDict = {}
#     for lat, long, cluster, primaryStreet, secondaryStreet, primaryFaciltyType, secondaryFaciltyType, primaryStreetSegId, secondaryStreetSegId in zip(dataset['LATITUDE'],dataset['LONGITUDE'],dataset['CLUSTER'],dataset['CLUSTERPRIMARYSTREET'],dataset['CLUSTERSECONDARYSTREET'],dataset['CLUSTERPRIMARYFACILITYTYPE'],dataset['CLUSTERSECONDARYFACILITYTYPE'], dataset['CLUSTERPRIMARYSTREETSEGID'], dataset['CLUSTERSECONDARYSTREETSEGID']):
    for lat, long, cluster, primaryStreet, secondaryStreet in zip(dataset['LATITUDE'],dataset['LONGITUDE'],dataset['CLUSTER'], dataset['CLUSTERPRIMARYSTREET'],dataset['CLUSTERSECONDARYSTREET']):
        if (cluster != -1):
                if (cluster in clusterDict.keys()):
                    thisCluster = clusterDict[cluster]
                    thisCluster.append([long,lat])
                    clusterDict[cluster] = thisCluster
                    totalAccidentNumberDict[cluster] +=1
                else:
                    clusterDict[cluster] = [[long,lat]]
                    primaryStreetDict[cluster] = str(primaryStreet)
                    secondaryStreetDict[cluster] = str(secondaryStreet)
#                     primaryFacilityTypeDict[cluster] = str(primaryFaciltyType)
#                     secondaryFacilityTypeDict[cluster] = str(secondaryFaciltyType)
#                     primaryStreetSegIdDict[cluster] = str(primaryStreetSegId)
#                     secondaryStreetSegIdDict[cluster] = str(secondaryStreetSegId)
                    totalAccidentNumberDict[cluster] = 0

    hulls = []
    clusters = []
    primaryStreets = []
    secondaryStreets = []
#     primaryFacilityTypes = []
#     secondaryFacilityTypes = []
#     primaryStreetSegIds = []
#     secondaryStreetSegIds = []
    totalAccidentNumbers = []
    clusterAreas = []
    for cluster in clusterDict.keys():
        clusters.append(cluster)
        thisHull = ConvexHull(clusterDict[cluster])
        hulls.append(thisHull)
        primaryStreets.append(primaryStreetDict[cluster])
        secondaryStreets.append(secondaryStreetDict[cluster])
#         primaryFacilityTypes.append(primaryFacilityTypeDict[cluster])
#         secondaryFacilityTypes.append(secondaryFacilityTypeDict[cluster])
#         primaryStreetSegIds.append(primaryStreetSegIdDict[cluster])
#         secondaryStreetSegIds.append(secondaryStreetSegIdDict[cluster])
        totalAccidentNumbers.append(totalAccidentNumberDict[cluster])
        clusterAreas.append(area(thisHull))
        
    clusters = pd.DataFrame(
            {
                "hull" : hulls,
                "cluster" : clusters,
                "primaryStreet" : primaryStreets,
                "secondaryStreet" : secondaryStreets,
#                 "primaryFacilityType" : primaryFacilityTypes, 
#                 "secondaryFacilityType" : secondaryFacilityTypes,
#                 "primaryStreetSegId" : primaryStreetSegIds,
#                 "secondaryStreetSegId" : secondaryStreetSegIds,
                "totalAccidents" : totalAccidentNumbers,
                "clusterArea" : clusterAreas
            })
        
    return clusters

def makeGeoJson(clusters):
    shapes = []

    for cluster, hull in zip(clusters['cluster'], clusters['hull']):
        outline = []
        for p in hull.vertices:
            outline.append(list(hull.points[p]))
        outline.append(list(hull.points[hull.vertices[0]]))
        shapes.append(outline)

    clusters["shape"] = shapes
    
    myGeoJson = {"type": "FeatureCollection","features":[]}   
    
    for index, row in clusters.iterrows():
        shape = row['shape']
        primaryStreet = row['primaryStreet']
        secondaryStreet = row['secondaryStreet']
#         primaryFacilityType = row['primaryFacilityType']
#         secondaryFaciltyType = row['secondaryFacilityType']
#         primaryStreetSegId = row['primaryStreetSegId']
#         secondaryStreetSegId = row['secondaryStreetSegId']
        totalAccidents = row['totalAccidents']
        clusterArea = row['clusterArea']
        thisFeature = {
            "type": "Feature",
            "properties": {
                "name": str(shape),
                "primaryStreet" : primaryStreet,
                "secondaryStreet" : secondaryStreet,
#                 "primaryFacilityType" : primaryFacilityType,
#                 "secondaryFaciltyType" : secondaryFaciltyType,
#                 "primaryStreetSegId" : primaryStreetSegId,
#                 "secondaryStreetSegId" : secondaryStreetSegId,
                "totalAccidents" : totalAccidents,
                "clusterArea" : clusterArea
             },
            "geometry" : {
                "type" : "Polygon",
                "coordinates" : [shape]
            }
        }
        myGeoJson['features'].append(thisFeature)

    geo_str = json.dumps(myGeoJson)
    return geo_str, clusters

def style_function(feature):
#     print(feature['properties'])
    # styling guide  https://python-visualization.github.io/folium/modules.html
     return {
            'fillColor': 'red',
            'fillOpacity' : .5,
            'color' : 'black'
        }
#     fillColor = ""
#     if (feature['properties']['primaryFacilityType'] == "None"):
#         return {
#             'fillColor': 'red',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Shared Lane"):
#         return {
#             'fillColor': 'orange',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Contraflow Bike Lane"):
#         return {
#             'fillColor': 'purple',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Existing Bike Lane"):
#         return {
#             'fillColor': 'blue',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Climbing Lane"):
#         return {
#             'fillColor': 'pink',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Cycle Track"):
#         return {
#             'fillColor': 'green',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     else:
#         return {
#             'fillColor': 'black',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }

In [6]:
# NOTES

# bikeData
# worst 3 hubs runCluster(bikeData, 0.0015, 25)
# more hubs runCluster(bikeData, 0.0014, 15)
# tightly packed runCluster(bikeData, 0.0005, 7)

# bikePedData
# runCluster(bikePedData, 0.0030, 3)

# bikeVehData
# bikeVehData(bikeVehData, 0.0010, 10)

# bikePedData

# clusteredData = runCluster(bikePedData, 0.01, 1)
# dcMap = makeMap(clusteredData)
# dcMap

# # bike vehicle data

# clusteredData = runCluster(bikeVehData, 0.0010, 10)
# dcMap = makeMap(clusteredData)
# dcMap

# bikeDataDeaths = bikeData[bikeData['FATAL_BICYCLIST'] > 0]
# bikeDataSmallDeaths = bikeDataDeaths [['LATITUDE','LONGITUDE']]
# bikeDataSmallDeaths.to_csv("data/cluster data/deathAccidents.csv")

In [69]:
dcMap = mapAll(bikeData)
dcMap

In [75]:
clusteredData = runCluster(bikeData, 0.0015, 25)
dcMap = makeMap(clusteredData)
dcMap

9.849336000010553
7.235664999973104
3.5729474999967734
4.631516499990511


In [76]:
clusteredData = runCluster(bikeData, 0.0014, 15)
dcMap = makeMap(clusteredData)
dcMap

2.25285699999285
5.844072000022974
9.139313000022675
17.484243999992938
11.180836499988644
6.8897730000055635
8.495266499999659
3.8638910000191258
2.7452095000188006
7.971775000006402
1.6949989999919197
4.4836004999949575
1.1709270000050926
7.36285449998299
2.235500499986586
2.0779870000072265


In [72]:
clusteredData = runCluster(bikeData, 0.0005, 7)
dcMap = makeMap(clusteredData)
dcMap

0.10999199999789343
0.141216000002828
0.11651150000107935
0.5876744999953786
0.10871699999866143
0.19644599999830514
0.1298184999999979
8.950000121784411e-05
0.04371750000064586
0.0938780000027856
0.013003999998107323
2.9999999919568835e-06
0.135476999998234
0.0831849999997057
0.10095149999962136
0.04443899999781621
0.26306000000322466
